# CDC State and County Policy Measures

* [COVID-19 State and County Policy Orders](https://healthdata.gov/dataset/covid-19-state-and-county-policy-orders)

In [ ]:
import pandas as pd
import numpy as np
import pycountry
import json
from datetime import datetime
from functools import reduce
import requests
from io import StringIO
import re
from bs4 import BeautifulSoup
import requests

In [ ]:
# papermill parameters
output_folder = "../output/"

In [ ]:
datasource_site = "https://healthdata.gov/dataset/COVID-19-State-and-County-Policy-Orders/gyqz-9u7n"
response = requests.get(datasource_site)
assert response.status_code == 200
url = "https://healthdata.gov/api/views/gyqz-9u7n/rows.csv?accessType=DOWNLOAD"

In [ ]:
# modified date
rpattern = re.search(r"\"Last Update\":\"(\w+ \d{1,2}, 202\d \d{2}:\d{2})\"", response.content.decode())
modified_date = rpattern.group(1)

In [ ]:
modified_date

In [ ]:
df = pd.read_csv(url)

In [ ]:
datetime.strptime('March 15, 2021', '%B %d, %Y')

In [ ]:
states = [subdivision.code.replace("US-", "") for subdivision in pycountry.subdivisions if subdivision.code.startswith("US-")]
df["ISO3166_1"] = "US"
df["ISO3166_2"] = df["state_id"]
df["LAST_UPDATE_DATE"] = modified_date
df["LAST_UPDATE_DATE"] = pd.to_datetime(df["LAST_UPDATE_DATE"].astype(str), format='%B %d, %Y %H:%M')
df['Last_Reported_Flag'] = df['LAST_UPDATE_DATE'].max() == df['LAST_UPDATE_DATE']
df = df[df["ISO3166_2"].isin(states)]

```sql
CREATE OR REPLACE TABLE POLICY_MEASURES (
  STATE_ID varchar(2),
  COUNTY varchar,
  FIPS_CODE float,
  POLICY_LEVEL varchar(32),
  DATE timestamp_ntz,
  POLICY_TYPE varchar(64),
  START_STOP varchar(5),
  COMMENTS varchar,
  SOURCE varchar,
  TOTAL_PHASE float,
  ISO3166_1 varchar(2),
  ISO3166_2 varchar(5),
  LAST_UPDATE_DATE timestamp_ntz,
  LAST_REPORTED_FLAG boolean
);
```

In [ ]:
df.dtypes

In [ ]:
df.to_csv(output_folder + "CDC_POLICY_MEASURES.csv", index=False, columns=['state_id',
 'county',
 'fips_code',
 'policy_level',
 'date',
 'policy_type',
 'start_stop',
 'comments',
 'source',
 'total_phases',
 'ISO3166_1',
 'ISO3166_2',
 'LAST_UPDATE_DATE',
 'Last_Reported_Flag'])